### 画像処理を並列化して高速化

サンプルコードは，カラー画像からグレイスケールにした画像を保存する処理

In [1]:
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import cv2
import numpy as np

def process(img: np.ndarray) -> np.ndarray:
    """グレースケールに変換する画像処理関数"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def process_image(file_path: Path, output_dir: Path):
    """指定された画像ファイルを処理して、指定されたディレクトリに保存する"""
    img = cv2.imread(str(file_path))
    img = process(img)
    output_path = output_dir / file_path.name
    cv2.imwrite(str(output_path), img)

def batch_process_images(input_dir: str, output_dir: str, max_workers: int = 4):
    """指定されたディレクトリ内のすべての画像をバッチ処理する"""
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    image_files = list(input_dir.glob('*.jpg'))  # 拡張子は適宜変更

    if not output_dir.exists():
        output_dir.mkdir(parents=True)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for file_path in image_files:
            executor.submit(process_image, file_path, output_dir)

# 使用例
input_directory = '../../sample_data/coco_sample_datasets/sample_coco_train2017/'  # 入力ディレクトリ
output_directory = './output/' # 出力ディレクトリ
batch_process_images(input_directory, output_directory, max_workers=128)
